In [ ]:
def read_list(list_path):
    list_data = []
    with open(list_path, "r") as f:
        for line in f.readlines():
            line = line.strip('\n') 
            list_data.append(line)  
    return list_data

### 0. generate list (slice, volume) of all data

In [ ]:
import os
import numpy as np
# import SimpleITK as sitk

def get_all_list(data_path,out_list):
    name_list = os.listdir(data_path)
    name_list.sort()
    print(len(name_list))
    file=open(out_list,'w')
    for line in name_list:
        line = line.split('.')[0]
        file.write(line+'\n')
    file.close()
    return name_list
## 3Dlist
# volume_data_path = "hecktor_datasets/hecktor_resampled/"
# volume_out_list = './volume_all.list'
# volume_list = get_all_list(volume_data_path,volume_out_list)
# or
volume_list = read_list('./Hecktor_slice/volume_all.list')
print(len(volume_list), volume_list)

## 2Dlist
slice_data_path = "Hecktor_slice/masks/"
slice_out_list = './all_slice.list'
slice_list = get_all_list(slice_data_path,slice_out_list)


## 1. Randomly split the volumes for train-test (from volum_all) or train-val (from volum_train)

In [ ]:
import random
random.shuffle(volume_list)
print("\nlist all rand: ",volume_list)
train_num = round(len(volume_list) * 0.8)
test_num = len(volume_list) - train_num
print(train_num, test_num)
list_train_rand = volume_list[:train_num]
list_test_rand = volume_list[train_num:]
print("list_train_rand:",list_train_rand,len(list_train_rand))        
print("\n list_test_rand:",list_test_rand,len(list_test_rand))

### 2. generate randP1_volume_train (test or val)

In [ ]:
out_train_list = './randP1_volume_train.list'
out_test_list = './randP1_volume_test.list'
print("train_list:",list_train_rand,len(list_train_rand))        
file=open(out_train_list,'w')
for line in list_train_rand:
    print(line)
    file.write(line+'\n')
file.close()

print("\n test_list:",list_test_rand,len(list_test_rand))
file_test=open(out_test_list,'w')
for line in list_test_rand:
    print(line)
    file_test.write(line+'\n')
file_test.close()

### 3. generate randP1_slice_all_train (test or val)

In [ ]:
out_train_list = './randP1_slice_all_train.list'
out_test_list = './randP1_slice_all_test.list'   

slices_all = []
slice_train_list = []
slice_test_list = []
with open("./all_slice.list", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')
        slices_all.append(line)
print(len(slices_all))
num = 0
for i in range(len(volume_list)):
    patient_name = volume_list[i]
    for j in range(len(slices_all)):
        if patient_name in slices_all[j]:
            if patient_name in list_train_rand: 
                slice_train_list.append(slices_all[j])
            else:
                slice_test_list.append(slices_all[j])
print(len(slice_train_list))
print(len(slice_test_list))

file=open(out_train_list,'w')
for line in slice_train_list:
    file.write(line+'\n')
file.close()

file_test=open(out_test_list,'w')
for line in slice_test_list:
    file_test.write(line+'\n')
file_test.close()

### 4.1 Generate a list of all slices with lesions (slice_nidus_all.list)

In [ ]:
import SimpleITK as sitk
root_dir = "hecktor_datasets/hecktor_resampled/"
mask_name = "ct_gtvt.nii.gz"
out_list = './slice_nidus_all.list'
volume_all_list = volume_list
all_nidus_list = []
for subsetindex in range(len(volume_all_list)):
    mask_path = root_dir + volume_all_list[subsetindex]  + '/' + (volume_all_list[subsetindex]) + mask_name
    mask = sitk.ReadImage(mask_path,sitk.sitkUInt8)
    mask_array = sitk.GetArrayFromImage(mask)
    print((volume_all_list[subsetindex]))
    
    n_sum = 0
    for n_slice in range(mask_array.shape[0]):
        if np.max(mask_array[n_slice,:,:]) != 0:
            n_sum +=1
            if n_sum == 1:
                first_slice = n_slice
                print("first slice:",first_slice)
            slice_num = n_slice + 1
            if len(str(slice_num)) == 1:
                new_slice_num = '00' + str(slice_num)
            elif len(str(slice_num)) == 2:
                new_slice_num = '0' + str(slice_num)
            else:
                new_slice_num = str(slice_num)
            maskpath = "{}_{}".format(volume_all_list[subsetindex],new_slice_num)
            all_nidus_list.append(maskpath)
    print("last slice:",first_slice+n_sum-1)
print("Done！")
print(len(all_nidus_list),all_nidus_list)

name_list = all_nidus_list
print(len(name_list))

file=open(out_list,'w')
for line in name_list:
    line = line.split('.')[0]
    print(line)
    file.write(line+'\n')
file.close()

## 4.2 generate randP1_slice_nidus_train (test or val)

In [ ]:
out_train_list = './randP1_slice_nidus_train.list'
out_test_list = './randP1_slice_nidus_test.list'   

slices_all = []
slice_train_list = []
slice_test_list = []
with open("./slice_nidus_all.list", "r") as f:
    for line in f.readlines():
        line = line.strip('\n') 
        slices_all.append(line)
print(len(slices_all))
num = 0
for i in range(len(volume_list)):
    patient_name = volume_list[i]
    for j in range(len(slices_all)):
        if patient_name in slices_all[j]:
            if patient_name in list_train_rand: 
                slice_train_list.append(slices_all[j])
            else:
                slice_test_list.append(slices_all[j])
print(len(slice_train_list))
print(len(slice_test_list))

file=open(out_train_list,'w')
for line in slice_train_list:
    file.write(line+'\n')
file.close()

file_test=open(out_test_list,'w')
for line in slice_test_list:
    file_test.write(line+'\n')
file_test.close()